In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
station = 'KCASANTA106'
staname = 'brommer'
now = datetime.now().date()
sdate = datetime(now.year-2,now.month-5,now.day-5).date()
datediff = (now-sdate).days

In [3]:
allwind = pd.read_pickle(f'{staname}_raw_{sdate}_{now-timedelta(days=1)}.pkl')

In [13]:
nnwind = allwind[allwind['Azimuth'].notnull()]

In [4]:
allwind[::len(allwind)//3]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
0,2020-05-21,12:04 AM,Spring,May,Overnight,53.8,East,90.0,0.0,0.0
262,2021-03-06,9:54 PM,Spring,Mar,Evening,47.9,NW,315.0,0.0,0.0
217,2022-01-06,6:07 PM,Winter,Jan,Evening,53.1,ENE,67.5,2.0,6.0


In [5]:
def dftostrFF(df):
    print(df.reset_index(drop=False).to_string(index=False))
def dftostrTT(df):
    print(df.reset_index(drop=True).to_string(index=True))
def dftostrTF(df):
    print(df.reset_index(drop=True).to_string(index=False))
def dftostrFT(df):
    print(df.reset_index(drop=False).to_string(index=True))

In [6]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}
revdir = [(value, key) for key, value in ordir.items()]
print('mapping of 16 angular wind directions to 16 compass points:\n')
print(pd.DataFrame(revdir,columns=['Azimuth','Compass']).to_string(index=False))
revord = dict(revdir)

mapping of 16 angular wind directions to 16 compass points:

 Azimuth Compass
     0.0   North
    22.5     NNE
    45.0      NE
    67.5     ENE
    90.0    East
   112.5     ESE
   135.0      SE
   157.5     SSE
   180.0   South
   202.5     SSW
   225.0      SW
   247.5     WSW
   270.0    West
   292.5     WNW
   315.0      NW
   337.5     NNW


In [157]:
def groupavg(df,grpcols,sortcols=None,ordcols=None):
    dfavg = df.groupby(grpcols)[['Speed','Azimuth']].mean().round(1)
    dfavg['Compass'] = (round(dfavg['Azimuth']/22.5,0)*22.5).map(revord) 
    dfavg.reset_index(drop=False,inplace=True)
    if ordcols != None:
        for i in range(len(grpcols)):
            dfavg[grpcols[i]] = pd.Categorical(dfavg[grpcols[i]], ordcols[i])
    if sortcols != None:
        return dfavg.sort_values(sortcols).reset_index(drop=True)
    return dfwavg

In [60]:
orderszn = ['Winter','Spring','Summer','Fall']
orderday = ['Overnight','Morning','Midday','Afternoon','Evening']
ordermth = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [166]:
groupavg(nnwind,['Season','Daypart'],['Season','Daypart'],[orderszn,orderday])

,Season,Daypart,Speed,Azimuth,Compass
0,Winter,Overnight,1.2,171.5,South
1,Winter,Morning,1.3,166.0,SSE
2,Winter,Midday,3.8,174.8,South
3,Winter,Afternoon,2.8,193.9,SSW
4,Winter,Evening,1.5,175.6,South
5,Spring,Overnight,1.1,135.6,SE
6,Spring,Morning,1.5,135.9,SE
7,Spring,Midday,4.4,160.2,SSE
8,Spring,Afternoon,4.9,153.3,SSE
9,Spring,Evening,2.8,153.9,SSE


In [164]:
print("unweighted average wind azimuth direction by season:\n")
groupavg(nnwind,['Season'],['Season'],[orderszn])

unweighted average wind azimuth direction by season:



,Season,Speed,Azimuth,Compass
0,Winter,2.0,175.8,South
1,Spring,2.7,146.0,SE
2,Summer,2.3,160.8,SSE
3,Fall,1.8,153.3,SSE


In [163]:
print("\n\nunweighted average wind azimuth direction by month:\n")
groupavg(nnwind,['Month'],['Month'],[ordermth])



unweighted average wind azimuth direction by month:



,Month,Speed,Azimuth,Compass
0,Jan,1.9,164.5,SSE
1,Feb,2.4,191.0,South
2,Mar,2.8,171.9,South
3,Apr,2.8,107.6,ESE
4,May,2.4,157.7,SSE
5,Jun,2.0,167.5,SSE
6,Jul,2.4,158.2,SSE
7,Aug,2.5,157.1,SSE
8,Sep,2.3,148.1,SSE
9,Oct,1.4,153.9,SSE


In [162]:
print("\n\nunweighted average wind azimuth direction by month:\n")
groupavg(nnwind,['Daypart'],['Daypart'],[orderday])



unweighted average wind azimuth direction by month:



,Daypart,Speed,Azimuth,Compass
0,Overnight,0.9,150.9,SSE
1,Morning,1.2,147.5,SSE
2,Midday,3.8,173.0,South
3,Afternoon,3.9,171.9,South
4,Evening,2.1,156.8,SSE


In [17]:
def groupwavg(df,groupcol,grouporder):
    wgtdir = (df['Speed']).multiply(df['Azimuth'])
    dfwgt = pd.concat([nnwind,pd.DataFrame(wgtdir,columns=['Wgtdir'])],axis=1)[[groupcol,'Speed','Wgtdir']]
    dfsum = dfwgt.groupby(groupcol).sum()
    wavgd = (dfsum.Wgtdir/dfsum.Speed)
    wintd = wavgd.astype(int)
    wclsd = round(wavgd/22.5,0)*22.5
    wcomp = wclsd.map(revord)
    dfwavg = pd.concat([pd.concat([wintd,wclsd],axis=1),wcomp],axis=1)
    dfwavg.columns = ['Azimuth','Closest','Compass']
    dfwavg = dfwavg.reset_index(drop=False)
    dfwavg[groupcol] = pd.Categorical(dfwavg[groupcol], grouporder)
    return dfwavg.sort_values(groupcol)

In [115]:
def multigrpwavg(df,grpcols,sortcols=None,ordcols=None):
    df['Wazm'] = df['Speed']*df['Azimuth']
    dfgsum = df.groupby(grpcols).sum()
    dfgsum['Wazm'] = (dfgsum['Wazm']/dfgsum['Speed']).astype(int)
    dfwavg = dfgsum[['Wazm']]
    dfwavg['Compass'] = (round(dfwavg['Wazm']/22.5,0)*22.5).map(revord) 
    dfwavg.reset_index(drop=False,inplace=True)
    if ordcols != None:
        for i in range(len(grpcols)):
            dfwavg[grpcols[i]] = pd.Categorical(dfwavg[grpcols[i]], ordcols[i])
    if sortcols != None:
        return dfwavg.sort_values(sortcols).reset_index(drop=True)
    return dfwavg

In [116]:
print('Speed-weighted Azimuth(Wazm),grouped by Season and Daypart, sorted by their explicit order:\n')
multigrpwavg(nnwind,['Season','Daypart'],['Season','Daypart'],[orderszn,orderday])

Speed-weighted Azimuth(Wazm),grouped by Season and Daypart, sorted by their explicit order:



,Season,Daypart,Wazm,Compass
0,Winter,Overnight,162,SSE
1,Winter,Morning,170,South
2,Winter,Midday,184,South
3,Winter,Afternoon,208,SSW
4,Winter,Evening,193,SSW
5,Spring,Overnight,141,SE
6,Spring,Morning,139,SE
7,Spring,Midday,172,South
8,Spring,Afternoon,172,South
9,Spring,Evening,179,South


In [117]:
print('Speed-weighted Azimuth(Wazm),grouped by Season and Daypart,sorted ascending by Wazm:\n')
multigrpwavg(nnwind,['Season','Daypart'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Season and Daypart,sorted ascending by Wazm:



,Season,Daypart,Wazm,Compass
0,Fall,Overnight,126,SE
1,Spring,Morning,139,SE
2,Spring,Overnight,141,SE
3,Fall,Evening,144,SE
4,Summer,Overnight,147,SSE
5,Summer,Morning,147,SSE
6,Fall,Morning,149,SSE
7,Summer,Evening,149,SSE
8,Winter,Overnight,162,SSE
9,Winter,Morning,170,South


In [126]:
print('Speed-weighted Azimuth(Wazm),grouped by Season,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Season'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Season,sorted ascending by Azimuth:



,Season,Wazm,Compass
0,Fall,164,SSE
1,Spring,166,SSE
2,Summer,166,SSE
3,Winter,185,South


In [128]:
print('Speed-weighted Azimuth(Wazm),grouped by Season,sorted explicitly by Season:\n')
multigrpwavg(nnwind,['Season'],'Season',[orderszn])

Speed-weighted Azimuth(Wazm),grouped by Season,sorted explicitly by Season:



,Season,Wazm,Compass
0,Winter,185,South
1,Spring,166,SSE
2,Summer,166,SSE
3,Fall,164,SSE


In [123]:
print('Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Month:\n')
multigrpwavg(nnwind,['Month'],'Month',[ordermth])

Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Month:



,Month,Wazm,Compass
0,Jan,174,South
1,Feb,204,SSW
2,Mar,189,South
3,Apr,132,SE
4,May,177,South
5,Jun,173,South
6,Jul,163,SSE
7,Aug,163,SSE
8,Sep,155,SSE
9,Oct,168,SSE


In [125]:
print('Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Month'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Month,sorted ascending by Azimuth:



,Month,Wazm,Compass
0,Apr,132,SE
1,Sep,155,SSE
2,Aug,163,SSE
3,Jul,163,SSE
4,Oct,168,SSE
5,Dec,172,South
6,Jun,173,South
7,Jan,174,South
8,May,177,South
9,Nov,180,South


In [130]:
print('Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted explicitly by Daypart:\n')
multigrpwavg(nnwind,['Daypart'],'Daypart',[orderday])

Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted explicitly by Daypart:



,Daypart,Wazm,Compass
0,Overnight,145,SE
1,Morning,150,SSE
2,Midday,180,South
3,Afternoon,178,South
4,Evening,162,SSE


In [129]:
print('Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted ascending by Azimuth:\n')
multigrpwavg(nnwind,['Daypart'],'Wazm')

Speed-weighted Azimuth(Wazm),grouped by Daypart,sorted ascending by Azimuth:



,Daypart,Wazm,Compass
0,Overnight,145,SE
1,Morning,150,SSE
2,Evening,162,SSE
3,Afternoon,178,South
4,Midday,180,South


In [19]:
print("difference in monthly average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(ordermth),(ordinalify(allwind,'Month',ordermth)).Azimuth.subtract((groupwavg(nnwind,'Month',ordermth)).Azimuth)],axis=1)
dfdiff.columns=['Month','Azimuth']
dftostrTF(dfdiff)
print("\n\ndifference in seasonal average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(orderszn),(ordinalify(allwind,'Season',orderszn)).Azimuth.subtract((groupwavg(nnwind,'Season',orderszn)).Azimuth)],axis=1)
dfdiff.columns=['Season','Azimuth']
dftostrTF(dfdiff)

difference in monthly average azimuth without vs with wind speed as weights:

Month  Azimuth
  Jan       32
  Feb       27
  Mar       -1
  Apr      -97
  May      -17
  Jun        4
  Jul      -15
  Aug      -32
  Sep      -29
  Oct      -27
  Nov       -8
  Dec       18


difference in seasonal average azimuth without vs with wind speed as weights:

Season  Azimuth
Winter       11
Spring      -21
Summer       -6
  Fall      -32


In [16]:
print("average wind speed by direction:\n")
dfs = allwind.groupby('Wind')['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Wind'] = pd.Categorical(dfs['Wind'], list(ordir.keys()))
dftostrTF(dfs.sort_values('Wind'))

print("ascending avg wind speed by direction:\n")
dftostrTF(dfs.sort_values('Speed'))

average wind speed by direction:

 Wind  Speed
North    1.8
  NNE    1.5
   NE    1.2
  ENE    1.8
 East    1.9
  ESE    2.4
   SE    1.8
  SSE    1.6
South    2.3
  SSW    3.1
   SW    2.7
  WSW    2.0
 West    1.4
  WNW    2.4
   NW    2.5
  NNW    2.0
  NaN    1.7
ascending avg wind speed by direction:

 Wind  Speed
   NE    1.2
 West    1.4
  NNE    1.5
  SSE    1.6
  NaN    1.7
   SE    1.8
  ENE    1.8
North    1.8
 East    1.9
  NNW    2.0
  WSW    2.0
South    2.3
  ESE    2.4
  WNW    2.4
   NW    2.5
   SW    2.7
  SSW    3.1
